### Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
from tic_plot import plot_grid
from IPython.display import clear_output
from tic_env import TictactoeEnv, OptimalPlayer
from performance_measures import M_opt, M_rand
from q_table import QTable
from policies import EpsilonGreedy, EpsilonGreedyDecreasingExploration

SEED = 1

$M_{\mathrm{opt}}$ measures the performance of $\pi$ against the optimal policy. <br>
To compute $M_{\mathrm{opt}}$, we run $\pi$ against `Opt(0)` for `N = 500` games for different random seeds. <br>
$\pi$ makes the 1st move in 250 games, and `Opt(0)` makes the 1st move in the rest. <br>
We count how many games $\pi$ wins ($N_{\mathrm{win}}$) and loses ($N_{\mathrm{loss}}$) and define $$M_{\mathrm{opt}} = \frac{N_{\mathrm{win}} - N_{\mathrm{loss}}}{N}$$.

$M_{\mathrm{rand}}$ measures the performance of against the random policy. <br>
To compute $M_{\mathrm{rand}}$, we repeat what we did for computing $M_{\mathrm{opt}}$ but by using `Opt(1)` instead of `Opt(0)`.

## Q-Learning combined with epsilon-greedy policy. 

At each time $t$, state $s_t$ is the board position, action $a_t$ is one of the available positions on the board and reward $r_t$. <br>
The reward only non-zero when the game ends where you get $r_t = 1$ if you win the game, $r_t = 1$ if you lose, and $r_t=0$ if it is a draw.

Q-Learning has 3 hyper-parameters: 
- learning rate $\alpha$
- discount factor $\gamma$
- and exploration level $\epsilon$ 

For convenience, we fix the learning rate at alpha = 0.05 and the discount factor at $\gamma$ = 0.99. <br>
We initialize all the Q-values at 0. <br>
If you are curious, you can explore the effect of $\alpha$, $\gamma$ and initial Q-values for yourself. <br>

In [ ]:
alpha = 0.05 # learning rate 
gamma = 0.99 # discount factor
epsilon = 0.1 # exploration rate

## 2.1 Learning from experts

In this section, you will study whether Q-learning can learn to play Tic Tac Toe by playing against `Opt(`$\epsilon_{opt}$`)` for some $\epsilon_{opt} \in [0; 1]$. <br> 
To do so, implement the Q-learning algorithm. To check the algorithm, run a Q-learning agent, with a fixed and arbitrary $\epsilon_{opt} \in [0; 1)$ against `Opt(0.5)` for 20'000 games. <br> 
Switch the 1st player after every game.

### Question 1

In [ ]:
env = TictactoeEnv()
Q_table = QTable()
rewards = []
num_trained_games = 0
av_rewards = []

policy = EpsilonGreedy(Q_table, epsilon=epsilon)
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
num_games = 20000
opponent_player = OptimalPlayer(0.5, player='O')
learning_player = 'X'
rewards = []

for itr in range(num_games):
    env.reset()
    state, end, _ = env.observe()

    opponent_player.player, learning_player = learning_player, opponent_player.player

    if opponent_player.player == 'X':
        opponent_move = opponent_player.act(state)
        state, end, _ = env.step(opponent_move)

    while not end:
        move = policy.act(state)
        next_state, end, _ = env.step(move)

        if (not end) and (env.current_player == opponent_player.player):
            opponent_move = opponent_player.act(next_state)
            next_state, end, _ = env.step(opponent_move)
        
        reward = env.reward(player=learning_player)
        
        Q_table[state][move] += alpha * (reward + gamma * max(Q_table[next_state].values()) - Q_table[state][move])

        state = next_state

    rewards.append(reward)
    if len(rewards) >= 250:
        av_rewards.append(np.mean(rewards))
        rewards = []
        clear_output(wait=True)
        plt.plot(av_rewards)
        plt.show()
    num_trained_games += 1
    env.reset()

In [ ]:
log_dir = './runs/Q1/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
with open(os.path.join(log_dir, 'av_rewards.pkl'), 'wb+') as f:
    pickle.dump(av_rewards, f)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')

plt.figure(figsize=(10, 5))
plt.plot(np.arange(0, num_trained_games, 250), av_rewards, "-")
plt.title(f"Average reward for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f}, Exploration rate: {epsilon:.2f}")
plt.ylabel("Avergage reward per 250 games")
plt.xlabel("Number of training games")
plt.grid(color="lightgrey")
plt.show()

## 2.1.1 Decreasing exploration

One way to make training more efficient is to decrease the exploration level $\epsilon$ over time. If we define $\epsilon(n)$ to be $\epsilon$ for game number $n$, then one feasible way to decrease exploration during training is to use
$$
\epsilon(n)=\max \left\{\epsilon_{\min }, \epsilon_{\max }\left(1-\frac{n}{n^{*}}\right)\right\}
$$
where $\epsilon_{\min }$ and $\epsilon_{\max }$ are the minimum and maximum values for $\epsilon$, respectively, and $n^{*}$ is the number of exploratory games and shows how fast $\epsilon$ decreases. For convenience, we assume $\epsilon_{\min }=0.1$ and $\epsilon_{\max }=0.8$; if you are curious, you can explore their effect on performance for yourself. Use $\epsilon(n)$ as define above and run different $Q$-learning agents with different values of $n^{*}$ against `Opt(0.5)` for 20'000 games - switch the 1st player after every game. Choose several values of $n^{*}$ from a reasonably wide interval between 1 to $40^{\prime} 000$ - particularly, include $n^{*}=1$.


### Question 2 and 3

In [ ]:
alpha = 0.05 # learning rate 
gamma = 0.99 # discount factor
epsilon_min, epsilon_max = 0.1, 0.8 # exploration rate

In [ ]:
av_rewards_dict = {}
m_opt_dict = {}
m_rand_dict = {}

In [ ]:
num_games = 20000

for n_star in (1, 5_000, 10_000, 20_000, 40_000):
    rewards = []
    opponent_player = OptimalPlayer(0.5, player='O')
    learning_player = 'X'
    av_rewards = []
    m_opt_list = []
    m_rand_list = []
    Q_table = QTable()
    policy = EpsilonGreedyDecreasingExploration(Q_table,
                                                epsilon_min=epsilon_min, 
                                                epsilon_max=epsilon_max, 
                                                n_star=n_star)
    env = TictactoeEnv()
    
    rewards = []
    num_trained_games = 0
    av_rewards = []

    random.seed(SEED)
    np.random.seed(SEED)

    for itr in range(num_games):
        env.reset()
        state, end, _ = env.observe()

        opponent_player.player, learning_player = learning_player, opponent_player.player

        if opponent_player.player == 'X':
            opponent_move = opponent_player.act(state)
            state, end, _ = env.step(opponent_move)

        while not end:
            policy.update_epsilon(itr)
            move = policy.act(state)
            next_state, end, _ = env.step(move)

            if (not end) and (env.current_player == opponent_player.player):
                opponent_move = opponent_player.act(next_state)
                next_state, end, _ = env.step(opponent_move)
            
            reward = env.reward(player=learning_player)
            
            Q_table[state][move] += alpha * (reward + gamma * max(Q_table[next_state].values()) - Q_table[state][move])

            state = next_state

        rewards.append(reward)
        if len(rewards) >= 250:
            av_rewards.append(np.mean(rewards))
            rewards = []
            clear_output(wait=True)
            plt.plot(av_rewards)
            plt.show()
            policy.set_epsilon(0)
            m_opt_list.append(M_opt(policy))
            m_rand_list.append(M_rand(policy))
        num_trained_games += 1
        env.reset()
    av_rewards_dict[n_star] = av_rewards
    m_opt_dict[n_star] = m_opt_list
    m_rand_dict[n_star] = m_rand_list

In [ ]:
log_dir = './runs/Q2/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
with open(os.path.join(log_dir, 'av_rewards_dict.pkl'), 'wb+') as f:
    pickle.dump(av_rewards_dict, f)

with open(os.path.join(log_dir, 'm_opt_dict.pkl'), 'wb+') as f:
    pickle.dump(m_opt_dict, f)

with open(os.path.join(log_dir, 'm_rand_dict.pkl'), 'wb+') as f:
    pickle.dump(m_rand_dict, f)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')

plt.figure(figsize=(10, 5))
for n_star, av_rewards in sorted(av_rewards_dict.items()):
    plt.plot(np.arange(0, num_trained_games, 250), av_rewards, "-", label=f"$n*={n_star}$")
plt.title(f"Average reward for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f},"+" $\epsilon_{min}$: "+str(epsilon_min)+" $\epsilon_{max}$: "+str(epsilon_max))
plt.ylabel("Avergage reward per 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')
plt.figure(figsize=(10, 5))
for n_star, m_opt_list in sorted(m_opt_dict.items()):
    plt.plot(np.arange(0, num_trained_games, 250), m_opt_list, "-", label=f"$n*={n_star}$")
plt.title("$M_{opt}$" + f" for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f},"+" $\epsilon_{min}$: "+str(epsilon_min)+" $\epsilon_{max}$: "+str(epsilon_max))
plt.ylabel("$M_{opt}$ for every 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')

plt.figure(figsize=(10, 5))
for n_star, m_rand_list in sorted(m_rand_dict.items()):
    plt.plot(np.arange(0, num_trained_games, 250), m_rand_list, "-", label=f"$n*={n_star}$")
plt.title("$M_{rand}$" + f" for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f},"+" $\epsilon_{min}$: "+str(epsilon_min)+" $\epsilon_{max}$: "+str(epsilon_max))
plt.ylabel("$M_{rand}$ for every 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

## 2.1.2 Good experts and bad experts
Choose the best value of $n^{*}$ that you found in the previous section. Run $Q$-learning against Opt $\left(\epsilon_{\mathrm{opt}}\right)$ for different values of $\epsilon_{\mathrm{opt}}$ for 20 '000 games $-$ switch the 1st player after every game. Choose several values of $\epsilon_{\mathrm{opt}}$ from a reasonably wide interval between 0 to $1-$ particularly, include $\epsilon_{\mathrm{opt}}=0$.

### Question 4

In [ ]:
alpha = 0.05 # learning rate 
gamma = 0.99 # discount factor
epsilon_min, epsilon_max = 0.1, 0.8 # exploration rate

In [ ]:
m_opt_per_epsilon = {}
m_rand_per_epsilon = {}

In [ ]:
n_star = 10000
num_games = 20000

for epsilon_opt in (0, 0.25, 0.50, 0.75, 1.0):
    rewards = []
    opponent_player = OptimalPlayer(epsilon_opt, player='O')
    learning_player = 'X'
    av_rewards = []
    m_opt_list = []
    m_rand_list = []
    Q_table = QTable()
    policy = EpsilonGreedyDecreasingExploration(Q_table,
                                                epsilon_min=epsilon_min, 
                                                epsilon_max=epsilon_max, 
                                                n_star=n_star)
    env = TictactoeEnv()
    
    num_trained_games = 0

    random.seed(SEED)
    np.random.seed(SEED)

    for itr in range(num_games):
        env.reset()
        state, end, _ = env.observe()

        opponent_player.player, learning_player = learning_player, opponent_player.player

        if opponent_player.player == 'X':
            opponent_move = opponent_player.act(state)
            state, end, _ = env.step(opponent_move)

        while not end:
            policy.update_epsilon(itr)
            move = policy.act(state)
            next_state, end, _ = env.step(move)

            if (not end) and (env.current_player == opponent_player.player):
                opponent_move = opponent_player.act(next_state)
                next_state, end, _ = env.step(opponent_move)
            
            reward = env.reward(player=learning_player)
            
            Q_table[state][move] += alpha * (reward + gamma * max(Q_table[next_state].values()) - Q_table[state][move])

            state = next_state

        rewards.append(reward)
        if len(rewards) >= 250:
            av_rewards.append(np.mean(rewards))
            rewards = []
            policy.set_epsilon(0)
            m_opt_list.append(M_opt(policy))
            m_rand_list.append(M_rand(policy))
            clear_output(wait=True)
            plt.subplot(2, 2, 1)
            plt.plot(m_opt_list)
            plt.subplot(2, 2, 2)
            plt.plot(m_rand_list)
            plt.subplot(2, 2, 3)
            plt.plot(av_rewards)
            plt.show()
        num_trained_games += 1
        env.reset()
    m_opt_per_epsilon[epsilon_opt] = m_opt_list
    m_rand_per_epsilon[epsilon_opt] = m_rand_list

In [ ]:
log_dir = './runs/Q4/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
with open(os.path.join(log_dir, 'm_opt_per_epsilon.pkl'), 'wb+') as f:
    pickle.dump(m_opt_per_epsilon, f)

with open(os.path.join(log_dir, 'm_opt_per_epsilon.pkl'), 'wb+') as f:
    pickle.dump(m_opt_per_epsilon, f)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')

plt.figure(figsize=(10, 5))
for e, m_opt_list in sorted(m_opt_per_epsilon.items()):
    plt.plot(np.arange(0, num_trained_games, 250), m_opt_list, "-", label="$\epsilon_{opt}$"+f"={e}")
plt.title("$M_{opt}$" + f" for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f}, $n^*$: {n_star}, "+"$\epsilon_{min}$: "+str(epsilon_min)+" $\epsilon_{max}$: "+str(epsilon_max))
plt.ylabel("$M_{opt}$ for every 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')

plt.figure(figsize=(10, 5))
for e, m_rand_list in sorted(m_rand_per_epsilon.items()):
    plt.plot(np.arange(0, num_trained_games, 250), m_rand_list, "-", label="$\epsilon_{opt}$"+f"={e}")
plt.title("$M_{rand}$" + f" for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f}, $n^*$: {n_star}, "+"$\epsilon_{min}$: "+str(epsilon_min)+" $\epsilon_{max}$: "+str(epsilon_max))
plt.ylabel("$M_{rand}$ for every 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

In [ ]:
max_m_opt = max(max(l) for l in m_opt_per_epsilon.values())
max_m_rand = max(max(l) for l in m_rand_per_epsilon.values())
max_m_opt, max_m_rand

## 2.2 Learning by self-practice

In this section, your are supposed to ask whether $Q$-learning can learn to play Tic Tac Toe by only playing against itself. 

For different values of $\epsilon \in[0,1)$, run a $Q$-learning agent against itself for 20'000 games - i.e. both players use the same set of $Q$-values and update the same set of $Q$-values.

### Question 7

In [ ]:
alpha = 0.05 # learning rate 
gamma = 0.99 # discount factor

In [ ]:
m_opt_against_itself = {}
m_rand_against_itself = {}

In [ ]:
n_star = 10000
num_games = 20000

for epsilon in (0, 0.25, 0.50, 0.75):
    m_opt_list = []
    m_rand_list = []
    Q_table = QTable()
    policy = EpsilonGreedy(Q_table, epsilon=epsilon)
    env = TictactoeEnv()
    
    num_trained_games = 0
    rewards_players1 = []
    avg_rewards_players1 = []
    random.seed(SEED)
    np.random.seed(SEED)

    for itr in range(num_games):
        env.reset()
        state0, end, _ = env.observe()

        current_player = 'X'
        next_player = 'O'

        move1 = policy.act(state0)
        state1, end, _ = env.step(move1)

        while not end:
            current_player, next_player = next_player, current_player

            move2 = policy.act(state1)
            state2, end, _ = env.step(move2)
            reward = env.reward(player = next_player)

            Q_table[state0][move1] += alpha * (reward + gamma * max(Q_table[state2].values()) - Q_table[state0][move1])

            state0 = state1.copy()
            state1 = state2.copy()
            move1 = move2

        reward = env.reward(player = current_player)
        Q_table[state0][move1] += alpha * (reward - Q_table[state0][move1])
        

        rewards_players1.append(env.reward(player = 'X'))
        if len(rewards_players1) >= 250:
            policy.set_epsilon(0)
            m_opt_list.append(M_opt(policy))
            m_rand_list.append(M_rand(policy))
            policy.set_epsilon(epsilon)
            avg_rewards_players1.append(np.mean(rewards_players1))
            rewards_players1 = []
            clear_output(wait=True)
            plt.subplot(2, 2, 1)
            plt.plot(m_opt_list)
            plt.subplot(2, 2, 2)
            plt.plot(m_rand_list)
            plt.show()

        num_trained_games += 1
        env.reset()
    m_opt_against_itself[epsilon] = m_opt_list
    m_rand_against_itself[epsilon] = m_rand_list

In [ ]:
log_dir = './runs/Q7/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
with open(os.path.join(log_dir, 'm_opt_against_itself.pkl'), 'wb+') as f:
    pickle.dump(m_opt_against_itself, f)

with open(os.path.join(log_dir, 'm_rand_against_itself.pkl'), 'wb+') as f:
    pickle.dump(m_rand_against_itself, f)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')

plt.figure(figsize=(10, 5))
for e, m_opt_list in sorted(m_opt_against_itself.items()):
    plt.plot(np.arange(0, num_trained_games, 250), m_opt_list, "-", label="$\epsilon$"+f"={e}")
plt.title("$M_{opt}$" + f" for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f}")
plt.ylabel("$M_{opt}$ for every 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')

plt.figure(figsize=(10, 5))
for e, m_rand_list in sorted(m_rand_against_itself.items()):
    plt.plot(np.arange(0, num_trained_games, 250), m_rand_list, "-", label="$\epsilon$"+f"={e}")
plt.title("$M_{rand}$" + f" for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f}")
plt.ylabel("$M_{rand}$ for every 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

In [ ]:
min(min(v.values()) for v in Q_table.values()), max(max(v.values()) for v in Q_table.values())

### Question 8

For rest of this section, use $\epsilon(n)$ in Equation 1 with different values of $n^*$ instead of fixing  $\epsilon$.

In [ ]:
alpha = 0.05 # learning rate 
gamma = 0.99 # discount factor
epsilon_min, epsilon_max = 0.1, 0.8 # exploration rate

In [ ]:
av_rewards_dict = {}
m_opt_dict = {}
m_rand_dict = {}

In [ ]:
num_games = 20000

for n_star in (1, 5_000, 10_000, 20_000, 40_000):
    Q_table = QTable()
    policy = EpsilonGreedyDecreasingExploration(Q_table, 
                                                epsilon_min=epsilon_min, 
                                                epsilon_max=epsilon_max, 
                                                n_star=n_star)
    env = TictactoeEnv()
    
    rewards_players1 = []
    avg_rewards_players1 = []
    m_opt_list = []
    m_rand_list = []
    num_trained_games = 0
    random.seed(SEED)
    np.random.seed(SEED)

    for itr in range(num_games):
        env.reset()
        state0, end, _ = env.observe()

        current_player = 'X'
        next_player = 'O'
        
        policy.update_epsilon(itr)
        move1 = policy.act(state0)
        state1, end, _ = env.step(move1)

        while not end:
            current_player, next_player = next_player, current_player

            move2 = policy.act(state1)
            state2, end, _ = env.step(move2)
            reward = env.reward(player = next_player)

            Q_table[state0][move1] += alpha * (reward + gamma * max(Q_table[state2].values()) - Q_table[state0][move1])

            state0 = state1.copy()
            state1 = state2.copy()
            move1 = move2

        reward = env.reward(player = current_player)
        Q_table[state0][move1] += alpha * (reward - Q_table[state0][move1])
        

        rewards_players1.append(env.reward(player = 'X'))
        if len(rewards_players1) >= 250:
            policy.set_epsilon(0)
            m_opt_list.append(M_opt(policy))
            m_rand_list.append(M_rand(policy))
            avg_rewards_players1.append(np.mean(rewards_players1))
            policy.update_epsilon(itr)
            rewards_players1 = []
            clear_output(wait=True)
            plt.subplot(2, 2, 1)
            plt.plot(m_opt_list)
            plt.subplot(2, 2, 2)
            plt.plot(m_rand_list)
            plt.show()
        num_trained_games += 1
        env.reset()
    av_rewards_dict[n_star] = avg_rewards_players1
    m_opt_dict[n_star] = m_opt_list
    m_rand_dict[n_star] = m_rand_list

In [ ]:
log_dir = './runs/Q8/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
with open(os.path.join(log_dir, 'm_opt_dict.pkl'), 'wb+') as f:
    pickle.dump(m_opt_dict, f)

with open(os.path.join(log_dir, 'm_rand_dict.pkl'), 'wb+') as f:
    pickle.dump(m_rand_dict, f)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')
plt.figure(figsize=(10, 5))
for n_star, m_opt_list in sorted(m_opt_dict.items()):
    plt.plot(np.arange(0, num_trained_games, 250), m_opt_list, "-", label=f"$n*={n_star}$")
plt.title("$M_{opt}$" + f" for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f},"+" $\epsilon_{min}$: "+str(epsilon_min)+" $\epsilon_{max}$: "+str(epsilon_max))
plt.ylabel("$M_{opt}$ for every 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-paper')

plt.figure(figsize=(10, 5))
for n_star, m_rand_list in sorted(m_rand_dict.items()):
    plt.plot(np.arange(0, num_trained_games, 250), m_rand_list, "-", label=f"$n*={n_star}$")
plt.title("$M_{rand}$" + f" for every 250 games during training\nLearning rate {alpha:.2f}, Discount factor: {gamma:.2f},"+" $\epsilon_{min}$: "+str(epsilon_min)+" $\epsilon_{max}$: "+str(epsilon_max))
plt.ylabel("$M_{rand}$ for every 250 games")
plt.xlabel("Number of training games")
plt.legend()
plt.grid(color="lightgrey")
plt.show()

### Save all Q-values as images and save model for debugging

In [ ]:
value2player = {0: '-', 1: 'X', -1: 'O'}
for i, s_tuple in enumerate(list(Q_table.keys())):
    try:
        plt.close()
        name = "".join(value2player[v] for v in s_tuple)
        s = np.array(s_tuple).reshape((3, 3))
        q = np.empty((3, 3))
        q[:] = np.NaN
        for coord, value in Q_table[s].items():
            q[coord] = value
        plot_grid(s, q)
        plt.clim(-1,1)
        plt.tight_layout()
        plt.savefig(f"images/debug_Q_table/{name}.png")
        #plt.show()
        plt.close()
    except AttributeError:
        pass

In [ ]:
with open('models/Q_table_15.27.pkl', 'wb') as f:
    pickle.dump(Q_table, f)

In [ ]:
with open(r'models/Q_table_Q8_04.05.2022_22.15.pkl', 'rb') as f:
    Q_table = pickle.load(f)
Q_table

In [ ]:
states = (
    ((0, 0, 0, 0, 0, 0, 0, 0, 0), 1),
    ((0, -1, -1, 0, 1, 1, 0, 0, 0), 1),
    ((0, -1, -1, 0, 1, 1, 1, 0, 0), -1),   
)

for i, (state, player) in enumerate(states):
    plt.close()
    s = np.array(state).reshape((3, 3))
    q = np.empty((3, 3))
    q[:] = np.NaN
    for coord, value in Q_table[s].items():
        q[coord] = value
    plot_grid(s, q, clim=(-1,1))
    plt.tight_layout()
plt.show()